In [ ]:
import sys
sys.path.append('..')

In [ ]:
!nvidia-smi

In [ ]:
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, GPT2Model
import transformers
import torch
from torch import nn
import copy
import time
from tqdm import tqdm
import gc

from light_attention.models.gpt2 import LightGPT2LMHeadModel, LightGPT2Model, LightGPT2Attention
from light_attention.profile import estimate_layer_memory, mem_usage

In [ ]:
torch.__version__, transformers.__version__, torch.device("cuda")

In [ ]:
RANDOM_SEED = 15835

# HuggingFace GPT2Model vs LightGPT2Model

# Memory

### GPT2-small

### Vanilla

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=12, n_layer=12, n_positions=1024, n_embd=768)
model = GPT2Model(configuration)
b = 4
seq = configuration.n_positions
emb = configuration.n_embd
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

### Light

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=12, n_layer=12, n_positions=1024, n_embd=768)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model = LightGPT2Model(configuration)
b = 4
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

### GPT2-medium

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=16, n_layer=24, n_positions=1024, n_embd=1024)
model = GPT2Model(configuration)
b = 4
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=16, n_layer=24, n_positions=1024, n_embd=1024)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model = LightGPT2Model(configuration)
b = 4
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

### GPT2-large

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=20, n_layer=36, n_positions=1024, n_embd=1280)
model = GPT2Model(configuration)
b = 4
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=20, n_layer=36, n_positions=1024, n_embd=1280)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model = LightGPT2Model(configuration)
b = 4
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

### GPT2-xl

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=25, n_layer=48, n_positions=1024, n_embd=1600)
model = GPT2Model(configuration)
b = 2
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

In [ ]:
torch.manual_seed(RANDOM_SEED)
configuration = GPT2Config(n_head=25, n_layer=48, n_positions=1024, n_embd=1600)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model = LightGPT2Model(configuration)
b = 4
seq = configuration.n_positions
x = torch.randint(0, configuration.vocab_size, size=(b,seq), device='cuda')
estimate_layer_memory(copy.deepcopy(model), x, device='cuda', input_shape=None)
torch.cuda.empty_cache()

# Correctness

In [ ]:
device = torch.device('cuda')

## FP32

In [ ]:
torch.manual_seed(RANDOM_SEED)
b = 4
seq = 1024
x1 = torch.randint(0, 50257, size=(b,seq), device='cuda')
configuration = GPT2Config(n_layer=1)
model1 = GPT2Model(configuration).to(device)
# model1.eval()
y1 = model1(x1).last_hidden_state
y1.cos().mean().backward()

#### LightGPT2Model through model class...

In [ ]:
torch.manual_seed(RANDOM_SEED)
b = 4
seq = 1024
x2 = torch.randint(0, 50257, size=(b,seq), device='cuda')
configuration = GPT2Config(n_layer=1)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model2 = LightGPT2Model(configuration).to(device)
# model2.eval()
y2 = model2(x2).last_hidden_state
y2.cos().mean().backward()

#### ... or through attention module substitution

In [ ]:
torch.manual_seed(RANDOM_SEED)
b = 4
seq = 1024
x2 = torch.randint(0, 50257, size=(b,seq), device='cuda')
configuration = GPT2Config(n_layer=1)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model2 = GPT2Model(configuration)
for i in range(len(model2.h)):
    weight_attn = model1.h[i].attn.c_attn.weight.detach()
    bias_attn = model1.h[i].attn.c_attn.bias.detach()
    weight_proj = model1.h[i].attn.c_proj.weight.detach()
    bias_proj = model1.h[i].attn.c_proj.bias.detach()
    model2.h[i].attn = LightGPT2Attention(configuration).cuda()
    model2.h[i].attn.c_attn.weight = nn.Parameter(weight_attn, requires_grad=True)
    model2.h[i].attn.c_attn.bias = nn.Parameter(bias_attn, requires_grad=True)
    model2.h[i].attn.c_proj.weight = nn.Parameter(weight_proj, requires_grad=True)
    model2.h[i].attn.c_proj.bias = nn.Parameter(bias_proj, requires_grad=True)
model2 = model2.cuda()
# model2.eval()
y2 = model2(x2).last_hidden_state
y2.cos().mean().backward()

In [ ]:
torch.allclose(x1, x2), \
torch.allclose(model1.h[0].attn.c_attn.weight, model2.h[0].attn.c_attn.weight), \
torch.allclose(model1.h[0].mlp.c_fc.weight, model2.h[0].mlp.c_fc.weight), \
torch.allclose(y1, y2), \
torch.allclose(model1.h[0].attn.c_attn.weight.grad, model2.h[0].attn.c_attn.weight.grad), \
torch.allclose(model1.h[0].mlp.c_fc.weight.grad, model2.h[0].mlp.c_fc.weight.grad)

In [ ]:
model1.h[0].attn.c_attn.weight.grad, model2.h[0].attn.c_attn.weight.grad

## Mixed Precision

In [ ]:
torch.manual_seed(RANDOM_SEED)
b = 4
seq = 1024
x1 = torch.randint(0, 50257, size=(b,seq), device='cuda')
configuration = GPT2Config(n_layer=1)
model1 = GPT2Model(configuration).to(device)
# model1.eval()

with torch.autocast(device_type='cuda', dtype=torch.float16):
    y1 = model1(x1).last_hidden_state
    y1.cos().mean().backward()

In [ ]:
torch.manual_seed(RANDOM_SEED)
b = 4
seq = 1024
x2 = torch.randint(0, 50257, size=(b,seq), device='cuda')
configuration = GPT2Config(n_layer=1)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model2 = LightGPT2Model(configuration).to(device)
# model2.eval()
with torch.autocast(device_type='cuda', dtype=torch.float16):
    y2 = model2(x2).last_hidden_state
    y2.cos().mean().backward()

In [ ]:
torch.allclose(x1, x2), \
torch.allclose(model1.h[0].attn.c_attn.weight, model2.h[0].attn.c_attn.weight), \
torch.allclose(model1.h[0].mlp.c_fc.weight, model2.h[0].mlp.c_fc.weight), \
torch.allclose(y1, y2), \
torch.allclose(model1.h[0].attn.c_attn.weight.grad, model2.h[0].attn.c_attn.weight.grad, atol=1e-6), \
torch.allclose(model1.h[0].mlp.c_fc.weight.grad, model2.h[0].mlp.c_fc.weight.grad)

In [ ]:
model1.h[0].attn.c_attn.weight.grad, model2.h[0].attn.c_attn.weight.grad

# Speed

## Forward

In [ ]:
device = torch.device('cuda')

In [ ]:
torch.manual_seed(RANDOM_SEED)

configuration = GPT2Config(n_layer=12)
model1 = GPT2Model(configuration).to(device)
# model1.eval()
seq = configuration.n_positions
emb = configuration.n_embd
samples = 100
b = 8
seq = 1024
xs = torch.randint(0, 50257, size=(samples, b,seq), device=device)

# fake run to allocate memory
with torch.no_grad():
    y = model1(xs[0])[0]
# cuda operations are asynchronous
torch.cuda.synchronize(device)

time1 = time.time()
with torch.no_grad():
    for x in tqdm(xs): 
        y = model1(x)[0]
        # cuda operations are asynchronous
        torch.cuda.synchronize(device)
time2 = time.time()
print(f'Forward pass takes {((time2-time1) / samples):.3} seconds on average. Computed for {samples} samples.')

In [ ]:
torch.manual_seed(RANDOM_SEED)

configuration = GPT2Config(n_layer=12)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = True
model2 = LightGPT2Model(configuration).to(device)
# model2.eval()
seq = configuration.n_positions
emb = configuration.n_embd
samples = 100
b = 8
seq = 1024
xs = torch.randint(0, 50257, size=(samples, b,seq), device=device)

# fake run to allocate memory
with torch.no_grad():
    y = model2(xs[0])[0]
# cuda operations are asynchronous
torch.cuda.synchronize(device)

time1 = time.time()
with torch.no_grad():
    for x in tqdm(xs): 
        y = model2(x)[0]
        # cuda operations are asynchronous
        torch.cuda.synchronize(device)
time2 = time.time()
print(f'Forward pass takes {((time2-time1) / samples):.3} seconds on average. Computed for {samples} samples.')

## Backward

In [ ]:
device = torch.device('cuda')

In [ ]:
torch.manual_seed(RANDOM_SEED)

configuration = GPT2Config(n_layer=10)
model1 = GPT2Model(configuration).to(device)
# model1.eval()
seq = configuration.n_positions
emb = configuration.n_embd
samples = 100
b = 4
seq = 1024
xs = torch.randint(0, 50257, size=(samples, b,seq), device=device)

# fake run to allocate memory
y = model1(xs[0])[0]
y.mean().backward()
# cuda operations are asynchronous
torch.cuda.synchronize(device)

time1 = time.time()
for x in tqdm(xs): 
    y = model1(x)[0]
    torch.cuda.synchronize(device)
    y.mean().backward()
    # cuda operations are asynchronous
    torch.cuda.synchronize(device)
time2 = time.time()
print(f'Forward pass takes {((time2-time1) / samples):.3} seconds on average. Computed for {samples} samples.')

In [ ]:
torch.manual_seed(RANDOM_SEED)

configuration = GPT2Config(n_layer=10)
configuration.use_dropmatmul = True
configuration.use_lightsoftmax = False
model2 = LightGPT2Model(configuration).to(device)
model2.eval()
seq = configuration.n_positions
emb = configuration.n_embd
samples = 100
b = 4
seq = 1024
xs = torch.randint(0, 50257, size=(samples, b,seq), device=device)

# fake run to allocate memory
y = model2(xs[0])[0]
y.mean().backward()
# cuda operations are asynchronous
torch.cuda.synchronize(device)

time1 = time.time()
for x in tqdm(xs): 
    y = model2(x)[0]
    torch.cuda.synchronize(device)
    y.mean().backward()
    # cuda operations are asynchronous
    torch.cuda.synchronize(device)
time2 = time.time()
print(f'Forward pass takes {((time2-time1) / samples):.3} seconds on average. Computed for {samples} samples.')